<a href="https://colab.research.google.com/github/ShayanPervez/tau-bench/blob/main/automated_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -qU langchain_community jq langchain_text_splitters langchain_huggingface sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.3/754.3 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [14]:
import os
from langchain_community.document_loaders import JSONLoader

# Folder containing JSON files
data_folder = "/content/data"

# List to collect all documents
all_documents = []

# Iterate through all files in the folder
for filename in os.listdir(data_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(data_folder, filename)

        loader = JSONLoader(
            file_path=file_path,
            jq_schema = 'to_entries[] | .value',
            text_content=False
        )

        documents = loader.load()
        all_documents.extend(documents)

# Now `all_documents` contains the combined loaded content from all JSON files
print(f"Loaded {len(all_documents)} documents.")


Loaded 20470 documents.


In [15]:
all_documents

[Document(metadata={'source': '/content/data/team_members.json', 'seq_num': 1}, page_content='{"team_id": "511b6d12-b81f-4f7c-8718-6b6596aafc4d", "worker_id": "04e74ccc-9243-4eee-8205-88d8e4114665", "role": "member", "joined_at": "2024-02-08", "left_at": null, "user_id": "c654f2e6-8d31-4e9f-84fc-72981fbbfaa9"}'),
 Document(metadata={'source': '/content/data/team_members.json', 'seq_num': 2}, page_content='{"team_id": "511b6d12-b81f-4f7c-8718-6b6596aafc4d", "worker_id": "4d492556-19f5-4f59-b494-29c700fa0868", "role": "member", "joined_at": "2024-12-25", "left_at": null, "user_id": "3118d3ac-226e-45a6-915b-f42cbe4fba6f"}'),
 Document(metadata={'source': '/content/data/team_members.json', 'seq_num': 3}, page_content='{"team_id": "511b6d12-b81f-4f7c-8718-6b6596aafc4d", "worker_id": "5a0d8ecd-8e2f-4b07-b9c2-c7440e4cbe5f", "role": "member", "joined_at": "2023-11-13", "left_at": null, "user_id": "90aea329-4842-42df-bc97-ead79343ce9c"}'),
 Document(metadata={'source': '/content/data/team_membe

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=80)
texts = text_splitter.split_documents(all_documents)

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [20]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 64.5 MB/s eta 0:00:00


In [63]:
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import os

# Create FAISS vector store
vectorstore = FAISS.from_texts(texts, embedding=hf)

# Save to local folder
save_path = "faiss_store"
vectorstore.save_local(save_path)


In [61]:
from google.colab import files
!zip -r faiss_store.zip faiss_store
files.download("faiss_store.zip")

updating: faiss_store/ (stored 0%)
updating: faiss_store/index.faiss


zip error: Interrupted (aborting)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
query = "status:suspended and role:hr_manager"
results = vectorstore.similarity_search(query, k=5)

for r in results:
    print(r.page_content)


{"user_id": "74061b5d-9cf8-4018-ba64-39c5c36eb09b", "organization_id": "2e786155-9aad-4686-93c5-901ed1a4b4ec", "worker_type": "employee", "status": "suspended"}
{"user_id": "94ac2916-107a-401f-8b89-7ae94f943a1d", "organization_id": "bfd2a770-f6c5-4734-a83b-cdad9f7ef5bc", "worker_type": "employee", "status": "suspended"}
{"user_id": "93d62845-c02a-4310-92e8-1150c09328a7", "organization_id": "deacb95a-ac0d-4f69-a8dd-10b4575aae7b", "worker_type": "employee", "status": "suspended"}
{"user_id": "d8db5f28-4223-4a63-babf-4d3c91c7faa4", "organization_id": "40858653-f35c-4b3c-ba36-a5614190e789", "worker_type": "employee", "status": "suspended"}
{"user_id": "32831d55-a6f5-435d-add7-972267781a5b", "organization_id": "a1534368-1d43-440a-b589-d42425d47a96", "worker_type": "employee", "status": "suspended"}


In [69]:
query = "give me details of 10 organization and it's name"
reteriever = vectorstore.as_retriever()
result = reteriever.invoke(query)
# Filter manually
for r in results:
    content = r.page_content
    print(content)


{"user_id": "74061b5d-9cf8-4018-ba64-39c5c36eb09b", "organization_id": "2e786155-9aad-4686-93c5-901ed1a4b4ec", "worker_type": "employee", "status": "suspended"}
{"user_id": "94ac2916-107a-401f-8b89-7ae94f943a1d", "organization_id": "bfd2a770-f6c5-4734-a83b-cdad9f7ef5bc", "worker_type": "employee", "status": "suspended"}
{"user_id": "93d62845-c02a-4310-92e8-1150c09328a7", "organization_id": "deacb95a-ac0d-4f69-a8dd-10b4575aae7b", "worker_type": "employee", "status": "suspended"}
{"user_id": "d8db5f28-4223-4a63-babf-4d3c91c7faa4", "organization_id": "40858653-f35c-4b3c-ba36-a5614190e789", "worker_type": "employee", "status": "suspended"}
{"user_id": "32831d55-a6f5-435d-add7-972267781a5b", "organization_id": "a1534368-1d43-440a-b589-d42425d47a96", "worker_type": "employee", "status": "suspended"}
